In [1]:
from G1_categorizer import worker

In [2]:
from nltk.corpus import stopwords
stops = set(stopwords.words("portuguese"))

In [4]:
%time print worker('http://g1.globo.com/economia/noticia/receita-com-vendas-de-remedios-cresce-131-em-2016-diz-interfarma.ghtml', stops)

vendas indústria farmacêutica brasileira cresceram somando r bilhões segundo levantamento realizado interfarma associação representa laboratórios farmacêuticos país partir dados ims health dez principais grupos farmacêuticos faturaram juntos r bilhões ano passado correspondendo mercado varejista segundo interfarma valor reflete preço lista medicamentos considerar eventuais descontos farmácias crescimento nominal receita vendas remédios país ficou acima inflação ficou índice preços consumidor amplo ipca segundo ibge medicamentos ficaram caros taxa elevada desde ficou categorias pressionaram inflação ano passado segundo levantamento receita vendas genéricos representam mercado subiu subiu categoria remédios líder faturamento classe analgésicos narcótico antipiréticos faturamento r bilhões total seguida antidepressivos estabilizadores humor vendas r bilhões crescimento comparação terceira posição ficaram preparações reguladoras colesterol triglicerídeos receita r bilhões
CPU times: user 6

In [5]:
from G1_categorizer import worker

In [6]:
%time worker('http://g1.globo.com/economia/noticia/receita-com-vendas-de-remedios-cresce-131-em-2016-diz-interfarma.ghtml', stops)

CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 640 ms


u'vendas ind stria farmac utica brasileira cresceram somando r bilh es segundo levantamento realizado interfarma associa representa laborat rios farmac uticos pa s partir dados ims health dez principais grupos farmac uticos faturaram juntos r bilh es ano passado correspondendo mercado varejista segundo interfarma valor reflete pre lista medicamentos considerar eventuais descontos farm cias crescimento nominal receita vendas rem dios pa s ficou acima infla ficou ndice pre consumidor amplo ipca segundo ibge medicamentos ficaram caros taxa elevada desde ficou categorias pressionaram infla ano passado segundo levantamento receita vendas gen ricos representam mercado subiu subiu categoria rem dios l der faturamento classe analg sicos n narc tico antipir ticos faturamento r bilh es total seguida est antidepressivos estabilizadores humor vendas r bilh es crescimento compara terceira posi ficaram prepara es reguladoras colesterol triglicer deos receita r bilh es'

In [7]:
import pandas as pd
from pandas import DataFrame

In [8]:
bag_df = pd.read_csv('bag_words.csv', sep=';')

In [9]:
bag_df.columns = ['link', 'categoria', 'texto']

In [10]:
bag_df.head()

,link,categoria,texto
0,http://g1.globo.com/economia/negocios/noticia/...,agro,vale anunciou manhã desta segundafeira venda a...
1,http://g1.globo.com/economia/noticia/receita-c...,ciencia-e-saude,faturamento venda remédios país somou r bilhõe...
2,http://g1.globo.com/economia/agronegocios/noti...,agro,etanol sobe estados df cai fica estável ap pe ...
3,http://g1.globo.com/bemestar/noticia/entenda-a...,ciencia-e-saude,nutella leva óleo palma composição europa aler...
4,http://g1.globo.com/distrito-federal/noticia/p...,ciencia-e-saude,procura vacina aumentou divulgada primeira mor...


In [11]:
bag_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 3 columns):
link         76 non-null object
categoria    76 non-null object
texto        75 non-null object
dtypes: object(3)
memory usage: 1.9+ KB


In [12]:
bag_df.describe()

,link,categoria,texto
count,76,76,75
unique,76,6,75
top,http://educacao.estadao.com.br/blogs/blog-da-t...,ciencia-e-saude,nada fazia parar falar fala desordenada repeti...
freq,1,20,1


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None)

In [22]:
train_data_features = vectorizer.fit_transform(list(bag_df.texto.dropna()))

In [23]:
train_data_features = train_data_features.toarray()

In [24]:
print train_data_features.shape

(75, 6146)


In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
forest = RandomForestClassifier(n_estimators = 100) 

In [30]:
bag_df_clean = bag_df.dropna(how='any')

In [32]:
forest = forest.fit( train_data_features, bag_df_clean.categoria )

In [36]:
from G1_categorizer import *

In [44]:
url = 'http://economia.estadao.com.br/noticias/geral,relator-diz-que-mudanca-ajuda-a-criar-empregos,70001636849'
texts = []
worker(url, '', texts)
text = texts[0][2]
print text

273
relator diz mudança ajuda criar empregos luiz carlos hauly psdbpr áreas ciência tecnologia podem ter incentivos via orçamento estado s paulo janeiro h proposta reforma tributária tramita congresso criação nova cpmf abarcaria iof usada permitir redução contribuição previdenciária empregado empregador segundo deputado luiz carlos hauly relator proposta alívio fundamental permitir aumento emprego alíquotas acima sobre folha empregador retidos empregado anime contratar diz desonerações incentivos hoje existentes acabariam automaticamente mudança quiserem benefícios vão ter negociar tudo novo comigo fazendo reforma estrutural sistema consertar país privilégios diz ideia é incentivos áreas específicas ciência tecnologia dados via orçamento meio tributos ideia é novo iva imposto seletivo compartilhados união estados municípios calibragem divisão bolo feita base fotografia média últimos anos primeiros cinco anos haveria perda ninguém é regra ouro relator espera conseguir apoio proposta nes

In [45]:
x = vectorizer.transform([text])
print x

  (0, 66)	1
  (0, 84)	1
  (0, 183)	1
  (0, 231)	1
  (0, 325)	1
  (0, 328)	2
  (0, 396)	1
  (0, 585)	1
  (0, 675)	1
  (0, 910)	2
  (0, 1053)	1
  (0, 1069)	2
  (0, 1275)	1
  (0, 1297)	1
  (0, 1364)	1
  (0, 1393)	1
  (0, 1483)	1
  (0, 1487)	1
  (0, 1571)	1
  (0, 1669)	1
  (0, 1740)	1
  (0, 1859)	1
  (0, 1910)	4
  (0, 2064)	1
  (0, 2065)	1
  :	:
  (0, 4585)	1
  (0, 4678)	3
  (0, 4808)	1
  (0, 4865)	1
  (0, 4908)	1
  (0, 4917)	1
  (0, 4928)	3
  (0, 4950)	1
  (0, 4978)	3
  (0, 5261)	1
  (0, 5274)	1
  (0, 5363)	1
  (0, 5375)	2
  (0, 5553)	2
  (0, 5587)	2
  (0, 5757)	1
  (0, 5758)	1
  (0, 5775)	1
  (0, 5833)	2
  (0, 5841)	1
  (0, 5882)	1
  (0, 5963)	2
  (0, 6065)	1
  (0, 6114)	2
  (0, 6140)	1


In [46]:
x = x.toarray()
print x

[[0 0 0 ..., 0 0 0]]


In [47]:
forest.predict(x)

array(['ciencia-e-saude'], dtype=object)